#### Friday, November 1, 2024

[Build a Chatbot](https://python.langchain.com/docs/tutorials/chatbot/)

mamba activate langchain

Using LMStudio to serve up hermes-3-llama-3.1-8b.

This all runs, but I do not understand why the model is sometimes referred to as "gpt-3.5-turbo" in the LMStudio logs.

Ok. Specify the model name when you instantiate the ChatOpenAI object. 

Wow. BUT if you do, then some of the cells fail to run.

#### Quickstart

In [1]:
import getpass
import os

In [2]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [4]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [5]:
# print(os.environ["LANGCHAIN_API_KEY"])
# print(os.environ["OPENAI_API_KEY"])

In [ ]:
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-4")
# model = ChatOpenAI(model="gpt-3.5-turbo")


# model = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)

model = ChatOpenAI(base_url="http://localhost:1234/v1", 
                   # model = "hermes-3-llama-3.1-8b",  # do not pass in an unrecognized model name ... 
                   api_key="lm-studio", 
                   temperature=0)


In [7]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello Bob! It's nice to meet you. How can I assist you today? Feel free to ask me anything you'd like help with.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 14, 'total_tokens': 43, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd1da575-dc25-4206-9e6d-700dc08f966b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 29, 'total_tokens': 43, 'input_token_details': {}, 'output_token_details': {}})

In [8]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='I don\'t have enough context to determine your actual name. If you would like, you can provide me with your name and I\'ll be happy to use it in our conversation. Otherwise, I\'ll just refer to you as "user" or by a placeholder name like "Alex" until you give me permission to use your real name.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 14, 'total_tokens': 82, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-227e845b-304a-4ab5-8106-592036c008e2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 68, 'total_tokens': 82, 'input_token_details': {}, 'output_token_details': {}})

In [9]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Based on our conversation so far, your name is Bob. Is there anything else I can help you with, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 39, 'total_tokens': 63, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-292057b7-aafb-4e8b-9749-3df909673001-0', usage_metadata={'input_tokens': 39, 'output_tokens': 24, 'total_tokens': 63, 'input_token_details': {}, 'output_token_details': {}})

#### Message Persistence

LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.

LangGraph comes with a simple in-memory checkpointer, which we use below. See its documentation for more detail, including how to use different persistence backends (e.g., SQLite or Postgres).

In [10]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

In [11]:
# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

In [12]:
# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a thread_id. This should look like:

In [13]:
config = {"configurable": {"thread_id": "abc123"}}

This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.

We can then invoke the application:

In [14]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Nice to meet you, Bob! How can I assist you today?


In [15]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. You mentioned that when we first started chatting. How else can I help or assist you today, Bob?


Great! Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [16]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have enough context to determine your actual name. If you would like, you can provide me with your name and I'll be happy to use it in our conversation. Otherwise, I'll just refer to you as "user" or by a placeholder name like "Alex" until you give me permission to use your real name.


However, we can always go back to the original conversation (since we are persisting it in a database)

In [17]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You've already told me your name is Bob. Is there anything else I can help with for you today, Bob?


For async support, update the call_model node to be an async function and use .ainvoke when invoking the application:

In [18]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't have enough context to determine your actual name. If you would like, you can provide me with your name and I'll be happy to use it in our conversation. Otherwise, I'll just refer to you as "user" or by a placeholder name like "Alex" until you give me permission to use your real name.


#### Prompt Templates

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

We can now update our application to incorporate this template:

In [20]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

We invoke the application in the same way:

In [21]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, matey! Jim be ye name, and welcome aboard me ship! What brings ye to these waters?


In [22]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arr, yer name be Jim, me hearty! Ye've told me so yerself, just a moment ago. Yer a friendly fella, ain't ya?


Awesome! Let's now make our prompt a little bit more complicated. Let's assume that the prompt template now looks something like this:

In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Note that we have added a new language input to the prompt. Our application now has two parameters-- the input messages and language. We should update our application's state to reflect this:

In [24]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [25]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿En qué puedo ayudarte hoy?


Note that the entire state is persisted, so we can omit parameters like language if no changes are desired:

In [26]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


#### Managing Conversation History

In [27]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [ ]:
# This fails IF we specify the model name when we instantiate ChatOpenAI ... !
# DO NOT RUN THIS CELL! I WANT TO KEEP THE ERROR MESSAGE FOR REFERENCE!
trimmer.invoke(messages)

NotImplementedError: get_num_tokens_from_messages() is not presently implemented for model cl100k_base. See https://platform.openai.com/docs/guides/text-generation/managing-tokens for information on how messages are converted to tokens.

In [ ]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt.

In [29]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Now if we try asking the model our name, it won't know it since we trimmed that part of the chat history:

In [30]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm not able to access or remember personal information like names. Could you please provide me with your name so I can assist you better?


But if we ask about information that is within the last few messages, it remembers:

In [31]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked for the sum of 2 and 2. The answer is 4.


#### Streaming

In [32]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Sure|,| here|'s| one| for| you|:

|Why| don|'t| scientists| trust| atoms|?

|Because| they| make| up| everything|!||